# MAMOB V2 — Trend Following Multi-Assets (CAC-like)

**Objectif** : construire un backtest *trend following* **long-only** multi-actifs, comparer **1 couple** vs **3 couples** de moyennes mobiles, et rendre l’évaluation **robuste** via :
- suppression du *look-ahead bias* (`shift(1)`),
- contrainte **fully invested (0 cash)** via normalisation + fallback,
- métriques **risk-adjusted** (CAGR, vol, Sharpe, MDD, turnover),
- tests de robustesse : **sensibilité aux paramètres** + (**option**) walk-forward IS/OOS.

> Ce notebook est volontairement **court** : il documente la logique “research” et les preuves principales.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.rcParams["figure.figsize"] = (11, 4)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 200)


## 1) Research question & protocole

### Research question
Est-ce qu’une stratégie trend-following simple basée sur moyennes mobiles, appliquée à un univers multi-actifs (CAC-like), produit une performance **risk-adjusted** supérieure à un benchmark **Buy & Hold equal-weight** ?

### Hypothèses testées
- **H1** : le signal de trend (MA_short > MA_long) améliore le Sharpe vs Buy & Hold.
- **H2** : la performance n’est pas un artefact de sur-optimisation : elle reste stable sur des variantes proches des paramètres (sensibilité).
- **H3** : 1 couple (ex: 20/100) peut être plus robuste/pratique que 3 couples (moins de degrés de liberté, turnover potentiellement plus faible), **et cela doit se voir dans les tests**.

### Règles de backtest (anti-pièges)
1) **No look-ahead** : on trade à *t+1* un signal calculé à *t* via `shift(1)`.
2) **No cash** : portefeuille toujours **100% investi** (normalisation quotidienne des poids).
3) **Comparaison complète** : on compare Trend vs Buy & Hold via rendement **et** risque (CAGR, vol, Sharpe, MDD) + turnover.

## 2) Données

Pour garder le notebook **rapide**, on illustre sur un sous-univers (8 titres).  
L’app Flask permet de sélectionner un univers plus large.

> ⚠️ Note : Yahoo Finance peut limiter les requêtes (rate-limit). Si ça échoue, relancer plus tard / changer de réseau / réduire l’univers.

In [ ]:
tickers = ["AI.PA","MC.PA","OR.PA","BNP.PA","SAN.PA","SU.PA","DG.PA","CAP.PA"]

raw = yf.download(
    tickers,
    period="15y",
    auto_adjust=False,
    progress=False,
    group_by="column"
)

prices = raw["Close"].dropna(how="all")
prices.tail()

In [ ]:
returns = prices.pct_change().fillna(0)
returns.tail()

## 3) Stratégie : signal MA + anti look-ahead

### Signal
- Signal haussier si `MA_short > MA_long` ⇒ 1
- Sinon ⇒ 0

### Pourquoi `shift(1)` ?
Sans `shift(1)`, on utilise la clôture de *t* pour décider et “trader” à *t* → *look-ahead bias*.  
Avec `shift(1)`, le signal calculé à *t* s’applique à *t+1* : backtest méthodologiquement propre.

In [ ]:
def ma_signal(prices: pd.DataFrame, short_w: int, long_w: int) -> pd.DataFrame:
    """Binary MA crossover signal, shifted by 1 day to avoid look-ahead."""
    ma_s = prices.rolling(short_w).mean()
    ma_l = prices.rolling(long_w).mean()
    sig = (ma_s > ma_l).astype(int)
    return sig.shift(1).fillna(0)

signal_20_100 = ma_signal(prices, 20, 100)
signal_20_100.tail()

## 4) Portefeuille **no cash** : normalisation + fallback

### Problème
Le signal donne une **exposition relative** par actif (0 ou 1 ; ou en tiers avec 3 couples).  
Mais pour obtenir un portefeuille, il faut imposer la contrainte :

\[
\sum_i w_i(t)=1
\]

### Solution (V2)
1) On normalise les expositions chaque jour.
2) Cas critique : si toutes les expositions sont nulles (tout bearish) ⇒ somme=0 ⇒ normalisation impossible.
   - Fallback retenu ici : **conserver les poids de la veille** (réduit le churn).
   - Au tout début (pas de veille) : **equal-weight**.

> Remarque : ce choix respecte l’hypothèse “no cash”.

In [ ]:
def normalize_weights_no_cash_with_fallback(exposure: pd.DataFrame) -> pd.DataFrame:
    """Normalize exposures to fully invested weights; fallback when all exposures are 0."""
    exposure = exposure.clip(lower=0)
    row_sum = exposure.sum(axis=1)
    w = exposure.div(row_sum.replace(0, np.nan), axis=0)
    w = w.ffill()  # keep last weights if sum=0
    w = w.fillna(1.0 / exposure.shape[1])  # initial equal-weight
    return w

weights_1 = normalize_weights_no_cash_with_fallback(signal_20_100)
weights_1.sum(axis=1).tail()

## 5) Backtest Trend (1 couple) + benchmark Buy & Hold

On calcule :
\[
r_p(t) = \sum_i w_i(t) r_i(t)
\]
et l’equity :
\[
Equity(t)=\prod_{u \le t}(1+r_p(u))
\]

Benchmark : Buy & Hold **equal-weight**, fully invested.

In [ ]:
def buy_hold_equal_weight(returns: pd.DataFrame) -> pd.Series:
    n = returns.shape[1]
    ew = np.repeat(1.0 / n, n)
    return (returns * ew).sum(axis=1)

port_ret_1 = (weights_1 * returns).sum(axis=1)
equity_1 = (1 + port_ret_1).cumprod()

bh_ret = buy_hold_equal_weight(returns)
bh_equity = (1 + bh_ret).cumprod()

pd.DataFrame({"Trend (20/100)": equity_1, "Buy&Hold EW": bh_equity}).dropna().plot(
    title="Equity curves — Trend 1-couple vs Buy & Hold"
)
plt.show()

## 6) 1 couple vs 3 couples (conviction en tiers)

### Construction “3 couples”
On prend 3 couples (fast/medium/slow), on calcule 3 signaux binaires, puis :

\[
exposure_i(t)=\frac{signal_1+signal_2+signal_3}{3} \in \{0,1/3,2/3,1\}
\]

Puis on **normalise** pour rester fully invested.

👉 Ici on compare **empiriquement** : Sharpe/MDD/turnover entre 1 et 3 couples.

In [ ]:
triple = [(12, 60), (24, 120), (48, 200)]

signals = [ma_signal(prices, s, l) for (s, l) in triple]
exposure_3 = sum(signals) / len(signals)

weights_3 = normalize_weights_no_cash_with_fallback(exposure_3)
port_ret_3 = (weights_3 * returns).sum(axis=1)
equity_3 = (1 + port_ret_3).cumprod()

pd.DataFrame({"Trend 1-couple": equity_1, "Trend 3-couples": equity_3, "Buy&Hold": bh_equity}).dropna().plot(
    title="Equity curves — 1-couple vs 3-couples (and Buy&Hold)"
)
plt.show()

## 7) KPIs “pro” : performance + risque + churn

On calcule :
- **Total return**
- **CAGR** (annualisé)
- **Vol annualisée**
- **Sharpe (rf=0)**
- **Max Drawdown (MDD)**
- **Turnover moyen** (proxy coûts / churn)

Ces métriques permettent une comparaison complète (et pas uniquement “ça monte”).

In [ ]:
def annualize_return(equity: pd.Series) -> float:
    if len(equity) < 2:
        return float("nan")
    total = equity.iloc[-1] / equity.iloc[0] - 1
    return (1 + total) ** (252 / len(equity)) - 1

def annualize_vol(ret: pd.Series) -> float:
    return float(ret.std() * np.sqrt(252))

def sharpe_ratio(ann_ret: float, ann_vol: float) -> float:
    if ann_vol == 0 or np.isnan(ann_vol):
        return float("nan")
    return float(ann_ret / ann_vol)

def max_drawdown(equity: pd.Series) -> float:
    peak = equity.cummax()
    dd = equity / peak - 1
    return float(dd.min())

def turnover_from_weights(weights: pd.DataFrame) -> pd.Series:
    return (weights - weights.shift(1).fillna(0)).abs().sum(axis=1)

def kpis(name: str, equity: pd.Series, ret: pd.Series, weights: pd.DataFrame) -> pd.Series:
    total = equity.iloc[-1] / equity.iloc[0] - 1
    ann = annualize_return(equity)
    vol = annualize_vol(ret)
    sh = sharpe_ratio(ann, vol)
    mdd = max_drawdown(equity)
    to_mean = float(turnover_from_weights(weights).mean())
    return pd.Series(
        {"Total Return": total, "CAGR": ann, "Vol": vol, "Sharpe": sh, "MaxDD": mdd, "Turnover_mean": to_mean},
        name=name
    )

kpi_table = pd.concat([
    kpis("Trend 1-couple (20/100)", equity_1, port_ret_1, weights_1),
    kpis("Trend 3-couples (12/60|24/120|48/200)", equity_3, port_ret_3, weights_3),
    kpis("Buy&Hold EW", bh_equity, bh_ret, pd.DataFrame(np.repeat(1/returns.shape[1], returns.shape[1]), index=returns.index, columns=returns.columns)),
], axis=1).T

# pretty display
disp = kpi_table.copy()
for col in ["Total Return","CAGR","Vol","MaxDD"]:
    disp[col] = (disp[col] * 100).round(2)
disp["Sharpe"] = disp["Sharpe"].round(2)
disp["Turnover_mean"] = disp["Turnover_mean"].round(3)
disp

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(kpi_table["Turnover_mean"], kpi_table["Sharpe"])
for name in kpi_table.index:
    plt.annotate(name, (kpi_table.loc[name,"Turnover_mean"], kpi_table.loc[name,"Sharpe"]))
plt.xlabel("Average turnover (proxy costs)")
plt.ylabel("Sharpe")
plt.title("Trade-off: Sharpe vs Turnover")
plt.show()

## 8) Robustness check : sensibilité aux paramètres autour de 20/100

Même si 20/100 marche, il faut vérifier qu’on n’est pas sur un point “chanceux”.  
On teste une petite grille autour de (20, 100) et on observe la distribution des Sharpe / MDD / turnover.

👉 Si 20/100 est proche du centre d’une zone de bons résultats, c’est un argument empirique fort.

In [ ]:
candidates = []
for s in [10, 15, 20, 25, 30]:
    for l in [80, 90, 100, 110, 120]:
        if s < l:
            candidates.append((s,l))

rows = []
for s,l in candidates:
    expo = ma_signal(prices, s, l)
    w = normalize_weights_no_cash_with_fallback(expo)
    r = (w * returns).sum(axis=1)
    eq = (1+r).cumprod()
    rows.append({
        "pair": f"{s}/{l}",
        "CAGR": annualize_return(eq),
        "Vol": annualize_vol(r),
        "Sharpe": sharpe_ratio(annualize_return(eq), annualize_vol(r)),
        "MaxDD": max_drawdown(eq),
        "Turnover": float(turnover_from_weights(w).mean())
    })

grid = pd.DataFrame(rows).sort_values("Sharpe", ascending=False).reset_index(drop=True)
grid.head(10)

In [ ]:
grid["Sharpe"].hist(bins=18)
plt.title("Sharpe distribution across MA candidates (sensitivity check)")
plt.xlabel("Sharpe")
plt.show()

## 9) (Option) Walk-forward IS/OOS — sélection défendable

Dans l’application Flask (V2), on implémente un **walk-forward** :
- split temporel : **70% IS** / **30% OOS**
- sélection du meilleur couple (ou triple) sur IS (ex: meilleur Sharpe IS)
- évaluation reportée sur OOS (vraie performance hors-échantillon)

👉 Dans ce notebook court, on ne ré-implémente pas tout le moteur walk-forward ; le code et les diagnostics sont dans l’app.

Ce que tu peux écrire dans un reporting :
- `IS` sert à choisir le paramètre
- `OOS` sert à juger la stratégie
- on ajoute aussi une **stabilité OOS** en 3 blocs (Sharpe bloc1/2/3) pour éviter qu’un bon résultat dépende d’une seule sous-période.

## 10) Difficultés rencontrées & solutions

1) **Look-ahead bias** → corrigé par `shift(1)`.

2) **Cash implicite** (somme des expositions < 1) → on impose **fully invested** via normalisation.

3) **Cas bearish global** (tous signaux à 0) → fallback **keep last weights** (réduit le churn) + equal-weight au début.

4) **Sur-optimisation des paramètres** → (dans l’app) walk-forward IS/OOS + diagnostics.

5) **Qualité / disponibilité des données Yahoo** → parfois rate-limit : limiter l’univers, relancer plus tard, batch + retries dans l’app.

## 11) Conclusion (résumé court)

On est passés d’un backtest simple (V1) à une version V2 plus “défendable” :
- backtest sans biais (shift(1))
- portefeuille **no cash** (normalisation + fallback)
- comparaison complète Trend vs Buy&Hold (CAGR/Vol/Sharpe/MDD/Turnover)
- preuves de robustesse : sensibilité aux paramètres (grid), et (dans l’app) walk-forward IS/OOS + diagnostics.

👉 Prochaines étapes naturelles : intégrer des coûts explicites (slippage/fees), et tester sur un univers plus large/périodes plus longues.